In [5]:
with open('files/file1.txt') as f:
    for row in f:
        print(row, end = '')
print('\n--------------------------')

with open('files/file2.txt') as f:
    for row in f:
        print(row, end = '')
print('\n--------------------------')

with open('files/file3.txt') as f:
    for row in f:
        print(row, end = '')

file1_line1
file1_line2
file1_line3
--------------------------
file2_line1
file2_line2
file2_line3
--------------------------
file3_line1
file3_line2
file3_line3

In [6]:
with open('files/file1.txt') as f1, open('files/file2.txt') as f2, open('files/file3.txt') as f3:
    print(f1.readlines())
    print(f2.readlines())
    print(f3.readlines())

['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']
['file3_line1\n', 'file3_line2\n', 'file3_line3']


In [19]:
from contextlib import contextmanager

@contextmanager
def open_file(f_name):
    print(f'opening {f_name}')
    f = open(f_name)
    try:
        yield f
    finally:
        print(f'closing {f_name}')
        f.close()


f_names = 'files/file1.txt', 'files/file2.txt', 'files/file3.txt'

exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

closing files/file2.txt
closing files/file1.txt
closing files/file3.txt


In [20]:
files = [enter() for enter in enters]

opening files/file1.txt
opening files/file2.txt
opening files/file3.txt


In [21]:
while True:
    try:
        rows = [next(f).strip('\n') for f in files]
    except StopIteration:
        break
    else:
        row = ','.join(rows)
        print(row)

file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3


In [22]:
for exit in exits[::-1]:
    exit(None, None, None)

closing files/file3.txt
closing files/file2.txt
closing files/file1.txt


In [24]:
class NestedContexts:
    def __init__(self, *contexts):
        self._enters = []
        self._exits = []
        self._values = []

        for ctx in contexts:
            self._enters.append(ctx.__enter__)
            self._exits.append(ctx.__exit__)

    def __enter__(self):
        for enter in self._enters:
            self._values.append(enter())
        return self._values

    def __exit__(self, exc_type, exc_value, exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_tb)
        return False
    

with NestedContexts(open_file('files/file1.txt'), 
                    open_file('files/file2.txt'), 
                    open_file('files/file3.txt')) as files:
    while True:
        try:
            rows = [next(f).strip('\n') for f in files]
        except StopIteration:
            break
        else:
            row = ','.join(rows)
            print(row)

opening files/file1.txt
opening files/file2.txt
opening files/file3.txt
file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3
closing files/file3.txt
closing files/file2.txt
closing files/file1.txt


In [25]:
contexts = [open_file(f_name) for f_name in f_names]

with NestedContexts(*contexts) as files:
    print('do work')

opening files/file1.txt
opening files/file2.txt
opening files/file3.txt
do work
closing files/file3.txt
closing files/file2.txt
closing files/file1.txt


In [ ]:
class NestedContexts:
    def __init__(self, *contexts):
        self._exits = []

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_tb)
        return False